### stock_info

In [15]:
# 종목기본정보 
# 종목번호(id) / 종목이름 / 시장구분

import pandas as pd

## data download : http://marketdata.krx.co.kr/mdi#document=040601
kospi_df = pd.read_csv("./data/kospi_data.csv", encoding="utf8") 
kosdaq_df = pd.read_csv("./data/kosdaq_data.csv", encoding="utf8")

kospi_df["시장구분"] = "KS"
kosdaq_df["시장구분"] = "KQ"

total_df = pd.concat([kospi_df, kosdaq_df], axis=0)
total_df = total_df[["종목코드", "기업명", "업종코드", "업종"]]
total_df = total_df.rename(columns={"종목코드":"stock_code", "기업명":"name", "업종코드":"business_code", "업종":"business"})

def code_to_string(code):
    code = str(code)
    if len(code) == 6:
        return code
    else:
        return "0"*(6-len(code)) + code
    
total_df["stock_code"] = total_df["stock_code"].map(lambda x : code_to_string(x))
total_df["business_code"] = total_df["business_code"].map(lambda x : code_to_string(x))

#### create table 

In [26]:
# 최초 실행시 table 생성

from db_info import db_info_get
from utils import db_connect
db_info = db_info_get("stock")

db_conn, cursor = db_connect(db_info)

cursor.execute("drop table if exists stock.stock_master")

sql = "CREATE table stock.stock_master(\
stock_code CHAR(6),\
name CHAR(20),\
business_code CHAR(6),\
business CHAR(100),\
TS TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,\
primary KEY(stock_code))\
 ENGINE=InnoDB\
 default CHARSET = utf8\
 collate = utf8_general_ci;"

cursor.execute(sql)

db_conn.commit()
db_conn.close()

#### data insert 

In [27]:
from utils import df_to_db
from db_info import db_info_get
from utils import db_connect

# database 접속 정보 로딩
db_info = db_info_get("stock")

df_to_db(total_df, "stock_master", db_info, if_exists="append")

#### data insert 확인 

In [24]:
db_conn, cursor = db_connect(db_info)
df = pd.read_sql("select * from stock.stock_master limit 5;" , db_conn)
print(df.head())

db_conn.commit()
db_conn.close()

  stock_code   name business_code        business                  TS
0     000020   동화약품        032102         의약품 제조업 2020-09-18 00:07:44
1     000040  KR모터스        033109  그외 기타 운송장비 제조업 2020-09-18 00:07:44
2     000050     경방        074701          종합 소매업 2020-09-18 00:07:44
3     000060  메리츠화재        116501             보험업 2020-09-18 00:07:44
4     000070  삼양홀딩스        116409          기타 금융업 2020-09-18 00:07:44
